In [1]:
d = '../data/pre/' # raw data directory

import os
import pandas as pd
import numpy as np
from collections import Counter
from time import time
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [ ]:
aC = pd.read_csv('../data/pre/app_categories.csv')

user_installedapps = pd.read_csv('%suser_installedapps.csv' % d)
ui_cates = pd.merge(user_installedapps, aC, on='appID', how='left')
ui_dum_cats = pd.get_dummies(ui_cates['appCategory'], prefix='insCat', dummy_na=True)
ui_cates = pd.concat([ui_cates, ui_dum_cats], axis=1)

In [ ]:
ui = ui_cates.groupby('userID').apply(lambda df: list(np.sum(df[[col for col in df.columns if 'insCat_' in col]].values, axis=0))).reset_index()

ui.columns = ['userID', 'insCat']

print('\nDev info\n',ui.head(), len(ui))

In [ ]:
cols = [col for col in ui_cates.columns if 'insCat_' in col]
ui_cats = pd.DataFrame(np.vstack(ui.pop('insAppCates').values))
ui_cats.columns = cols
ui = pd.concat([ui, ui_cats], axis=1)

In [ ]:
user_app_actions = pd.read_csv('%suser_app_actions.csv' % d)
uact_cate = pd.merge(user_app_actions, aC, on='appID', how='left')
uact_dum_cats = pd.get_dummies(uact_cate['appCategory'], prefix='actCat', dummy_na=True)
uact_cate = pd.concat([uact_cate, uact_dum_cats], axis=1)

In [ ]:
# uact_cate['installTime_d'] = uact_cate.installTime.map(lambda x: int(str(x)[0:2]))
uact_cate = uact_cate.groupby('userID').apply(lambda df: list(np.sum(df[[col for col in df.columns if 'actCat_' in col]].values, axis=0))).reset_index()
uact_cate.columns = ['userID', 'actCat']

In [ ]:
ua_cols = [c for c in uact_dum_cats.columns if 'actCat_' in col]
ua_cats = pd.DataFrame(np.vstack(uact_cate.pop('actCat').values))
ua_cats.columns = ua_cols
uact_cate = pd.concat([uact_cate, ua_cats], axis=1)

In [ ]:
te_df = pd.read_csv('../data/pre/new_generated_test.csv', index_col=0)
te_df = pd.merge(te_df, ui, on='userID', how='left')
te_df = pd.merge(te_df, uact_cate, on='userID', how='left')
tr_df = pd.read_csv('../data/pre/new_generated_train.csv', index_col=0)
tr_df = pd.merge(tr_df, ui, on='userID', how='left')
tr_df = pd.merge(tr_df, uact_cate, on='userID', how='left')
tr_df.to_csv('../data/pre/complete_train.csv', index=None)
te_df.to_csv('../data/pre/complete_test.csv', index=None)

In [ ]:
al = pd.concat([tr_df.appCategory, te_df.appCategory, aC.appCategory], ignore_index=True)
adAppCate = pd.DataFrame({'appCategory': al.values})
adAppCate = pd.get_dummies(adAppCate.appCategory)
tr_adapp = adAppCate[:len(tr_df)]
tr_adapp.columns = ['adAppCat_%d'%d for d in tr_adapp.columns]
te_adapp = adAppCate[len(tr_df):len(tr_df)+len(te_df)]
te_adapp.columns = ['adAppCat_%d'%d for d in te_adapp.columns]
te_df = pd.concat([te_df, te_adapp], axis=1)
tr_df = pd.concat([tr_df, tr_adapp], axis=1)

In [ ]:
tr = pd.read_csv('../data/pre/complete_train.csv')
te = pd.read_csv('../data/pre/complete_test.csv')

In [ ]:
len(tr), len(te)

In [97]:
# data

'''
fs = ['appCategory', 'positionID', 'positionType', 'creativeID', 'appID', 'adID',
            'advertiserID', 'camgaignID', 'sitesetID', 'connectionType',
            'residence', 'age', 'hometown', 'haveBaby', 'telecomsOperator',
            'gender', 'education', 'clickTime_h', 'clickTime_d', 'weekDay',
            'marriageStatus', 'appPlatform', 'clickTime_m', 'userID']
'''


fs = ['appCategory', 'positionID', 'positionType', 'creativeID', 'appID', 'adID',
            'advertiserID', 'camgaignID', 'sitesetID', 'connectionType',
            'residence', 'age', 'hometown', 'haveBaby', 'telecomsOperator',
            'gender', 'education', 'clickTime_h', 'clickTime_d', 'weekDay',
            'marriageStatus', 'appPlatform', 'clickTime_m', 'userID']
fs_ac = [c for c in te.columns if 'adAppCat_' in c]
fs_ui = [c for c in te.columns if 'insCat_' in c]
fs_ua = [c for c in te.columns if 'actCat_' in c]

# ==================================================

a_fs = fs+fs_ac+fs_ui+fs_ua
print(len(a_fs))

tr_x = tr_df[a_fs].values
tr_y = tr_df['label'].values



te_x = te_df[a_fs].values

# xgboost

import xgboost as xgb  
from numpy import sort
from xgboost import plot_importance
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import log_loss
from matplotlib import pyplot


gbm = xgb.XGBClassifier(max_depth=args.md, max_delta_step=1, silent=False, n_estimators=args.ne, 
                        learning_rate=0.3, objective='binary:logistic', 
                        min_child_weight = 1, scale_pos_weight = 1,  
                        subsample=0.8, colsample_bytree=0.8,
                       ).fit(tr_x, tr_y, eval_set=[(va_x, va_y)], 
                        eval_metric='logloss', verbose=True)

predict_probas = gbm.predict_proba(te_x)[:,1]
save_preds(predict_probas)

va_y_pred = gbm.predict_proba(va_x)[:,1]

logloss = log_loss(va_y, va_y_pred)
print('Logloss: ', logloss)
va_y_pred_class = gbm.predict(va_x)

s= classification_report(va_y, va_y_pred_class)
print(s)

va_y_pred = (va_y_pred > 0.5).astype('int32')
s= classification_report(va_y, va_y_pred)
s_rocauc= roc_auc_score(va_y, va_y_pred)
print(s, '\n', s_rocauc)

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-97-bdb7e4097deb>, line 64)